# Ensemple CB recommender based on Sentence Transformers

In [1]:
 %load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../lib')

import torch
import numpy as np
import pandas as pd

import pytorch_common.util as pu

import logging
import data as dt

import recommender as rc

<Figure size 432x288 with 0 Axes>

## Setup

In [3]:
pu.LoggerBuilder().on_console().build()

<RootLogger root (INFO)>

In [4]:
pu.set_device_name('gpu')

In [5]:
pu.get_device()

device(type='cuda', index=0)

In [6]:
torch.cuda.is_available()

True

In [7]:
torch.__version__

'1.11.0'

## Evaluación

In [8]:
def load(name): return pd.read_json(f'../datasets/movie_{name}_embedding_bert.json')
def emb_rec(emb, name): return rc.DistanceMatrixRecommender(emb, f'{name}_embedding')

In [9]:
embeddings_by_name   = {name: load(name) for name in ['genres', 'overview', 'tags', 'title']}
recommenders_by_name = {name: emb_rec(emb, name) for name, emb in embeddings_by_name.items()}
emb_recommenders     = list(recommenders_by_name.values())

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

Building Distances Matrix:   0%|          | 0/18515 [00:00<?, ?it/s]

In [10]:
item_index = 10

In [11]:
recommender_group = rc.RecommenderGroup(emb_recommenders)

In [12]:
recommender_group.recommend(item_index).show()


Recommender: genres
Item


,id,title,imdb_id,image
10,8360,Shrek 2,298148,


Recommendations


,index,distance,id,title,imdb_id,image
0,10,0.000000,8360,Shrek 2,298148,
1,2860,0.017834,2081,"Little Mermaid, The",97757,
2,2739,0.019455,56152,Enchanted,461770,
3,2858,0.027384,1907,Mulan,120762,
4,7187,0.041949,88468,Alpha and Omega,1213012,



Recommender: overview
Item


...

,index,distance,id,title,imdb_id,image
0,10,0.000000,8360,Shrek 2,298148,
1,2733,0.193913,45517,Cars,317219,
2,1916,0.223048,53121,Shrek the Third,413267,
3,3,0.226049,4306,Shrek,126029,
4,19,0.233767,78499,Toy Story 3,435761,



Recommender: title
Item


,id,title,imdb_id,image
10,8360,Shrek 2,298148,


Recommendations


,index,distance,id,title,imdb_id,image
0,10,0.000000,8360,Shrek 2,298148,
1,2954,0.109930,78637,Shrek Forever After,892791,
2,1916,0.122315,53121,Shrek the Third,413267,
3,3,0.128482,4306,Shrek,126029,
4,14344,0.264189,64249,Shrek the Halls,897387,


In [98]:
recommender = rc.EnsembleWeightedDistanceMatrixRecommender(
    emb_recommenders,
    weights = { 'tags': 0.25, 'overview': 0.3, 'genres': 0.2, 'title': 0.25}
)

In [99]:
recommender.recommend(item_index, k=20).show()


Recommender: ['genres', 'overview', 'tags', 'title']
Item


,id,title,imdb_id,image
10,8360,Shrek 2,298148,


Recommendations


,distance,title,imdb_id,image
0,0.000000,Shrek 2,298148,
1,0.045675,Shrek the Third,413267,
2,0.057980,Shrek,126029,
3,0.060789,Shrek Forever After,892791,
4,0.090587,Toy Story 2,120363,
5,0.092630,Aladdin,103639,
6,0.093295,Shrek the Halls,897387,
7,0.097207,Toy Story,114709,
8,0.097843,Toy Story 3,435761,
9,0.100463,Tangled,398286,
